# Noise model

In [1]:
import sys
sys.path.insert(0, "../source/")

from FOM_advectiondiffusion import FOM_advectiondiffusion
from FOM_advectiondiffusion_steadystate import FOM_advectiondiffusion_steadystate
from Drone import Drone
from Parameter import Parameter
from Brain import Brain
from NoiseModel import NoiseModel
from Prior import Prior

import dolfin as dl
import matplotlib.pyplot as plt
import numpy as np
import time
import scipy.linalg as la
import scipy.sparse as sparse
import scipy.sparse.linalg as sla
import math
import scipy.stats as stats

In [2]:
tStart_notebook = time.time()

## User Settings

In [76]:
meshDim = 50
# meshDim=50 is probably too coarse but for writing the code it should suffice, I'm impatient

n_para = 1

eval_mode = "point-eval" # because it's faster for testing code

dt = 1e-4

## Setup

In [77]:
fom = FOM_advectiondiffusion_steadystate(meshDim=meshDim)

grid_t = np.arange(0, 1+dt, dt)

para_true = np.ones((n_para,))
m = fom.assemble_forcing(para=para_true)
sol_steadystate = fom.solve_steady(forcing=m)

# set default flying parameters
flying_parameters = {
                "radius" : 0.25,
                "center" : np.array([0.75/2, 0.55/2]),
                "speed" : 0.2 # 0.3
            }

drone = Drone(fom, flying_parameters=flying_parameters, eval_mode=eval_mode, grid_t=grid_t)
pos, __ = drone.get_trajectory(flying_parameters=flying_parameters)

prior = Prior(covar = np.eye(n_para), mean = np.zeros((n_para,)))
noise_model = NoiseModel(grid_t=grid_t, c_scaling=1e+3, c_diffusion = 0.01)
brain = Brain(fom=fom, drone=drone, prior=prior, noise_model=noise_model)

## Measurements

In [78]:
measurements = drone.measure(flightpath=pos, state=sol_steadystate)

In [88]:
slicer = 5
noise_model_coarse = NoiseModel(grid_t=grid_t[::slicer], c_scaling=1e+3, c_diffusion = 0.01)

In [89]:
measurements.T @ noise_model.mass_matrix @ measurements

8.037811476659929e-07

In [90]:
measurements[::slicer].T @ noise_model_coarse.mass_matrix @ measurements[::slicer]

8.037817421314292e-07

In [ ]:
please stop here

In [7]:
measurements.T @ noise_model.apply_covar_inv(d=measurements)

33.27525283716568

In [8]:
measurements[::slicer].T @ noise_model_coarse.apply_covar_inv(d=measurements[::slicer])

17.279194550686952

In [70]:
dt = 1e-2
grid_t = np.arange(0, 1+dt, dt)
grid_t_expanded = np.arange(-dt, 1+dt+dt, dt)
#grid_t_expanded[int(0.5/dt):int(1.5/dt)+1]

drone = Drone(fom, flying_parameters=flying_parameters, eval_mode=eval_mode, grid_t=grid_t)
pos, __ = drone.get_trajectory(flying_parameters=flying_parameters)
measurements = drone.measure(flightpath=pos, state=sol_steadystate)

noise_model_expanded = NoiseModel(grid_t=grid_t_expanded, c_scaling=1e+3, c_diffusion = 0.01)
measurements_expanded = np.hstack([np.zeros(int(dt/dt),), measurements, np.zeros(int(dt/dt),)])
measurements_expanded.T @ noise_model_expanded.apply_covar_inv(d=measurements_expanded)

5129.11817542103

In [66]:
grid_t_expanded.shape

(3,)

In [28]:
grid_t_expanded = np.arange(-0.5, 1.5+dt, dt)

In [29]:
grid_t_expanded[500:1501]

array([4.4408921e-16, 1.0000000e-03, 2.0000000e-03, ..., 9.9800000e-01,
       9.9900000e-01, 1.0000000e+00])

In [30]:
noise_model_expanded = NoiseModel(grid_t=grid_t_expanded, c_scaling=1e+3, c_diffusion = 0.01)

In [31]:
measurements_expanded = np.hstack([np.zeros(500,), measurements, np.zeros(500,)])

In [32]:
measurements_expanded.T @ noise_model_expanded.apply_covar_inv(d=measurements_expanded)

3055006.042887622

In [49]:
slicer = 10
grid_t_expanded_sliced = grid_t_expanded[::slicer]
noise_model_sliced = NoiseModel(grid_t=grid_t_expanded_sliced, c_scaling=1e+3, c_diffusion = 0.01)
measurements_expanded[::slicer].T @ noise_model_sliced.apply_covar_inv(d=measurements_expanded[::slicer])

3169.933826687188

In [46]:
measurements_expanded[::slicer].shape

(1001,)

In [19]:
grid_t_expanded = np.arange(-0.1, 1.1+dt, dt)
noise_model_expanded = NoiseModel(grid_t=grid_t_expanded, c_scaling=1e+3, c_diffusion = 0.01)
measurements_expanded = np.hstack([np.zeros(100,), measurements, np.zeros(100,)])
measurements_expanded.T @ noise_model_expanded.apply_covar_inv(d=measurements_expanded)

3055006.042887622